In [1]:
# 11. Detaylı İstatistiksel Analiz ve Güven Aralıkları (CI)
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# --- 1. Veriyi Okuma ---
print("--- Veri Yükleme Aşaması ---")
url = 'https://www.dropbox.com/s/luoopt5biecb04g/SATILIK_EV1.xlsx?dl=1'
try:
    veri = pd.read_excel(url)
    print(f"Veri başarıyla yüklendi. Toplam Satır: {len(veri)}, Toplam Sütun: {len(veri.columns)}")
except Exception as e:
    print(f"Hata: Veri yüklenemedi! Detay: {e}")

# --- 2. Değişken Hazırlığı ---
# Bağımsız değişkenler (X) ve Bağımlı değişken (y) tanımlanıyor
X = veri[['Oda_Sayısı', 'Net_m2', 'Katı', 'Yaşı']]
y = veri['Fiyat']

print(f"\nBağımsız Değişkenler: {list(X.columns)}")
print(f"Hedef Değişken: Fiyat")

# --- 3. Veriyi Bölme ---
# %80 Eğitim, %20 Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Eğitim seti boyutu: {X_train.shape[0]} örnek")

# --- 4. Statsmodels İçin Sabit Terim (Intercept) Ekleme ---
# Matematiksel olarak y = b0 + b1*x denklemi için b0 (const) ekliyoruz.
X_train_sm = sm.add_constant(X_train)
print("> Modele sabit terim (bias/intercept) eklendi.")

# --- 5. OLS Modelini Kurma ve Eğitme ---
print("> OLS (En Küçük Kareler) modeli hesaplanıyor...\n")
model_sm = sm.OLS(y_train, X_train_sm).fit()

# --- 6. Sonuçları Yazdırma ---
print("="*80)
print("İSTATİSTİKSEL MODEL ANALİZ RAPORU")
print("="*80)
print(model_sm.summary())

# --- 7. Güven Aralıklarını ve P-Değerlerini Şık Bir Tabloyla Gösterme ---
print("\n" + "="*40)
print("KATSAYI ANALİZİ VE GÜVEN ARALIKLARI")
print("="*40)

# Güven aralıkları ve p-değerlerini birleştirip daha okunabilir yapalım
summary_df = pd.concat([model_sm.params, model_sm.pvalues, model_sm.conf_int()], axis=1)
summary_df.columns = ['Katsayı (Coef)', 'P-Değeri (P>|t|)', '[0.025', '0.975]']

print(summary_df.round(4))

print("\nNOT: P-Değeri < 0.05 olan değişkenler 'İstatistiksel Olarak Anlamlı' kabul edilir.")

--- Veri Yükleme Aşaması ---
Veri başarıyla yüklendi. Toplam Satır: 191, Toplam Sütun: 6

Bağımsız Değişkenler: ['Oda_Sayısı', 'Net_m2', 'Katı', 'Yaşı']
Hedef Değişken: Fiyat
Eğitim seti boyutu: 152 örnek
> Modele sabit terim (bias/intercept) eklendi.
> OLS (En Küçük Kareler) modeli hesaplanıyor...

İSTATİSTİKSEL MODEL ANALİZ RAPORU
                            OLS Regression Results                            
Dep. Variable:                  Fiyat   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.726
Method:                 Least Squares   F-statistic:                     101.3
Date:                Sat, 28 Feb 2026   Prob (F-statistic):           3.16e-41
Time:                        20:41:20   Log-Likelihood:                -827.75
No. Observations:                 152   AIC:                             1666.
Df Residuals:                     147   BIC:                             1681.
Df Model:                        

In [4]:
# 1. Genel Bakış (Sadece Sayısal Sütunlar)
print("\n" + "="*20 + " 1. SAYISAL ÖZET (SADECE NUMERIC) " + "="*20)
print(veri.describe())
print("-" * 74)

# 2. Tüm Sütunlar (Kategorik/Metinsel Veriler Dahil)
print("\n" + "="*20 + " 2. TÜM SÜTUNLAR (KATEGORİK DAHİL) " + "="*19)
# Not: 'unique', 'top' ve 'freq' değerleri sadece kategorik sütunlar için görünür.
print(veri.describe(include="all"))
print("-" * 74)

# 3. Spesifik Veri Tipleri (Sadece Float/Ondalıklı Sütunlar)
print("\n" + "="*20 + " 3. SADECE FLOAT (ONDALIKLI) TİPLER " + "="*18)
try:
    print(veri.describe(include=[float]))
except ValueError:
    print("Bilgi: Veri setinde 'float' tipinde sütun bulunmamaktadır.")
print("-" * 74)

# 4. Özelleştirilmiş Yüzdelikler (Uç Değer Analizi İçin)
print("\n" + "="*20 + " 4. ÖZEL YÜZDELİK DİLİMLER (%5, %50, %95) " + "="*14)
# Uç değerleri (outliers) yakalamak için çok kullanışlıdır.
print(veri.describe(percentiles=[.05, .50, .95]))
print("-" * 74)

# 5. Okunaklı ve Temiz Görünüm
print("\n" + "="*20 + " 5. YUVARLANMIŞ VE TEMİZ GÖRÜNÜM " + "="*22)
# Karmaşık sayıları (örn: 123.45678) 2 basamağa indirger.
print(veri.describe().round(2))
print("=" * 74 + "\n")


==================== 1. SAYISAL ÖZET (SADECE NUMERIC) ====================
       Unnamed: 0        Fiyat  Oda_Sayısı      Net_m2        Katı        Yaşı
count    191.0000   191.000000  191.000000  191.000000  191.000000  191.000000
mean      95.0000   649.476440    2.340314   85.183246    3.931937   12.753927
std       55.2811   111.064174    0.721401   16.482620    3.752974    8.374596
min        0.0000   450.000000    1.000000   40.000000    0.000000    0.000000
25%       47.5000   575.000000    2.000000   80.000000    0.000000    4.000000
50%       95.0000   650.000000    2.000000   90.000000    3.000000   16.000000
75%      142.5000   700.000000    3.000000   95.000000    7.000000   20.000000
max      190.0000  1175.000000    3.000000  125.000000   11.000000   24.000000
--------------------------------------------------------------------------

==================== 2. TÜM SÜTUNLAR (KATEGORİK DAHİL) ===================
       Unnamed: 0        Fiyat  Oda_Sayısı      Net_m2        